### Jupyter Notebook: Integration Tests for `apply_strategy_rules`

#### Cell 1: Setup (Prerequisite)

We'll assume the `apply_strategy_rules` function is already defined from the Layer 1 notebook. If you're starting a new notebook, make sure to run the cell that defines the function first.

In [1]:
import pandas as pd
import numpy as np
import operator

# --- Ensure the function to be tested is defined in your environment ---
# (You can copy the function definition here if running in a new notebook)

def apply_strategy_rules(features, rules, config):
    """
    Applies a list of filtering rules to a features DataFrame.

    Args:
        features (pd.DataFrame): The DataFrame containing all calculated features.
        rules (list): A list of dictionaries, where each dict defines a filtering rule.
        config (dict): The configuration dictionary, used for dynamic thresholds.

    Returns:
        pd.Series: A boolean Series (mask) indicating which rows pass all rules.
    """
    # Start with a mask that is True for all rows. We will progressively filter it.
    final_mask = pd.Series(True, index=features.index)
    
    # Map operator strings to actual Python operator functions for flexibility
    op_map = {
        '>': operator.gt,
        '<': operator.lt,
        '>=': operator.ge,
        '<=': operator.le,
        '==': operator.eq,
        '!=': operator.ne
    }

    for rule in rules:
        op_func = op_map[rule['operator']]
        
        # --- Rule Type 1: Comparing two columns ---
        if 'column_A' in rule and 'column_B' in rule:
            mask = op_func(features[rule['column_A']], features[rule['column_B']])
        
        # --- Rule Type 2: Comparing a column to a value ---
        elif 'column' in rule:
            # Determine the value to compare against
            if 'value' in rule:
                value = rule['value']
            elif 'value_from_config' in rule:
                value = config[rule['value_from_config']]
            else:
                raise ValueError(f"Rule missing 'value' or 'value_from_config': {rule}")
            
            mask = op_func(features[rule['column']], value)
            
        else:
            raise ValueError(f"Invalid rule format: {rule}")
        
        # Combine the mask for this rule with the final mask using a logical AND
        final_mask &= mask
            
    return final_mask
  

print("Setup complete. Ready for Layer 2 tests.")

Setup complete. Ready for Layer 2 tests.


#### Cell 2: Test 2.1 - Multiple Conditions on a Single Column (Range Test)

This is the classic "between" test. We'll verify that two rules applied to the same column correctly narrow down the results.


In [2]:
print("--- Test 2.1: Combining Rules on a Single Column (Range Test) ---")
# Arrange: Create a predictable DataFrame
test_df = pd.DataFrame({'A': [-1.0, 0.0, 2.5, 4.9, 5.0, 10.0]})
rules = [
    {'column': 'A', 'operator': '>', 'value': 0},
    {'column': 'A', 'operator': '<', 'value': 5}
]
config = {}

print("\nOriginal DataFrame:")
print(test_df)
print(f"\nApplying Rules: (A > 0) AND (A < 5)")

# Act: Apply the rules
mask = apply_strategy_rules(test_df, rules, config)
result_df = test_df[mask]

print("\nResulting DataFrame:")
print(result_df)

# Assert: Check if the output is exactly what we expect
expected_indices = [2, 3] # Indices where A is between 0 and 5
try:
    assert result_df.index.tolist() == expected_indices
    print("\n[SUCCESS]: Combining rules on a single column works as expected.")
except AssertionError:
    print("\n[FAILURE]: Range test produced an incorrect result.")
    print("Expected indices:", expected_indices)
    print("Got indices:", result_df.index.tolist())

--- Test 2.1: Combining Rules on a Single Column (Range Test) ---

Original DataFrame:
      A
0  -1.0
1   0.0
2   2.5
3   4.9
4   5.0
5  10.0

Applying Rules: (A > 0) AND (A < 5)

Resulting DataFrame:
     A
2  2.5
3  4.9

[SUCCESS]: Combining rules on a single column works as expected.


#### Cell 3: Test 2.2 - Multiple Conditions on Different Columns

Here, we'll check that the `AND` logic works correctly when filtering based on two different features.

In [3]:
print("\n--- Test 2.2: Combining Rules on Different Columns ---")
# Arrange: Create a DataFrame with rows that satisfy one, the other, both, or neither condition
test_df = pd.DataFrame({
    'price_z_score': [-2.0, -1.8,  1.5,  1.2],
    'volume_slope':  [ 0.1, -0.1,  0.2, -0.3]
})
rules = [
    {'column': 'price_z_score', 'operator': '<', 'value': -1.5},
    {'column': 'volume_slope',  'operator': '>', 'value': 0}
]
config = {}

print("\nOriginal DataFrame:")
print(test_df)
print("\nApplying Rules: (price_z_score < -1.5) AND (volume_slope > 0)")

# Act: Apply the rules
mask = apply_strategy_rules(test_df, rules, config)
result_df = test_df[mask]

print("\nResulting DataFrame:")
print(result_df)

# Assert: Only the row at index 0 satisfies both conditions
expected_indices = [0]
try:
    assert result_df.index.tolist() == expected_indices
    print("\n[SUCCESS]: Combining rules on different columns works as expected.")
except AssertionError:
    print("\n[FAILURE]: Filtering on different columns produced an incorrect result.")
    print("Expected indices:", expected_indices)
    print("Got indices:", result_df.index.tolist())


--- Test 2.2: Combining Rules on Different Columns ---

Original DataFrame:
   price_z_score  volume_slope
0           -2.0           0.1
1           -1.8          -0.1
2            1.5           0.2
3            1.2          -0.3

Applying Rules: (price_z_score < -1.5) AND (volume_slope > 0)

Resulting DataFrame:
   price_z_score  volume_slope
0           -2.0           0.1

[SUCCESS]: Combining rules on different columns works as expected.


#### Cell 4: Test 2.3 - The "Stress Test" (Combination of All Rule Types)

This is the final integration test. We'll combine all three types of rules (static value, config value, and column-vs-column) to ensure they all work together seamlessly.

In [4]:
print("\n--- Test 2.3: Stress Test - Combining All Rule Types ---")
# Arrange: Create a DataFrame and a config
test_df = pd.DataFrame({
    'low_slope':    [0.06, 0.04, 0.08, 0.09],
    'high_slope':   [0.05, 0.08, 0.07, 0.12],
    'r_squared':    [0.6,  0.8,  0.9,  0.4]
})
config = {'min_r_squared': 0.7}
rules = [
    # Rule 1: Static value
    {'column': 'low_slope', 'operator': '>', 'value': 0.05},
    
    # Rule 2: Value from config
    {'column': 'r_squared', 'operator': '>', 'value_from_config': 'min_r_squared'},
    
    # Rule 3: Column-vs-column
    {'column_A': 'high_slope', 'operator': '>', 'column_B': 'low_slope'}
]

print("\nOriginal DataFrame:")
print(test_df)
print(f"\nConfig: {config}")
print("\nApplying Rules:")
print("1. low_slope > 0.05")
print("2. r_squared > 0.7")
print("3. high_slope > low_slope")

# Act: Apply the rules
mask = apply_strategy_rules(test_df, rules, config)
result_df = test_df[mask]

print("\nResulting DataFrame:")
print(result_df)

# Assert: Let's trace the logic.
# Row 0: Fails rule 2 (0.6 < 0.7) and rule 3 (0.05 < 0.06)
# Row 1: Fails rule 1 (0.04 < 0.05)
# Row 2: Fails rule 3 (0.07 < 0.08)
# Row 3: Fails rule 2 (0.4 < 0.7)
# Hmm, let's re-check the logic.
# Row 0: Fails rule 2 and 3. OUT
# Row 1: Fails rule 1. OUT
# Row 2: Fails rule 3. OUT
# Row 3: Passes rule 1 (0.09>0.05), Fails rule 2 (0.4<0.7), Passes rule 3 (0.12>0.09). OUT because rule 2 fails.
# It seems my data was bad. Let's make a row that passes.
test_df = pd.DataFrame({
    'low_slope':    [0.06, 0.04, 0.08, 0.09, 0.08],
    'high_slope':   [0.05, 0.08, 0.07, 0.04, 0.09],
    'r_squared':    [0.6,  0.8,  0.9,  0.4,  0.9]
})
print("\nOops, my test data was bad. Let's try again with a row that should pass.")
print("New Original DataFrame:")
print(test_df)
# Row 4 should pass: low_slope (0.08>0.05), r_squared (0.9>0.7), high_slope (0.09>0.08)
mask = apply_strategy_rules(test_df, rules, config)
result_df = test_df[mask]
print("\nResulting DataFrame (Second Attempt):")
print(result_df)


expected_indices = [4]
try:
    assert result_df.index.tolist() == expected_indices
    print("\n[SUCCESS]: The stress test combining all rule types works as expected.")
except AssertionError:
    print("\n[FAILURE]: The stress test produced an incorrect result.")
    print("Expected indices:", expected_indices)
    print("Got indices:", result_df.index.tolist())


--- Test 2.3: Stress Test - Combining All Rule Types ---

Original DataFrame:
   low_slope  high_slope  r_squared
0       0.06        0.05        0.6
1       0.04        0.08        0.8
2       0.08        0.07        0.9
3       0.09        0.12        0.4

Config: {'min_r_squared': 0.7}

Applying Rules:
1. low_slope > 0.05
2. r_squared > 0.7
3. high_slope > low_slope

Resulting DataFrame:
Empty DataFrame
Columns: [low_slope, high_slope, r_squared]
Index: []

Oops, my test data was bad. Let's try again with a row that should pass.
New Original DataFrame:
   low_slope  high_slope  r_squared
0       0.06        0.05        0.6
1       0.04        0.08        0.8
2       0.08        0.07        0.9
3       0.09        0.04        0.4
4       0.08        0.09        0.9

Resulting DataFrame (Second Attempt):
   low_slope  high_slope  r_squared
4       0.08        0.09        0.9

[SUCCESS]: The stress test combining all rule types works as expected.


### *Note: I intentionally left in my thought process for debugging the test data in the final cell. It's a realistic example of how even when the code is right, the test setup needs to be right too!*

With these Layer 2 tests passing, you now have a very high degree of confidence that your dynamic rule engine is robust, correct, and ready for any strategy you can think of.